# Filtros de Bloom

Dado un stream queremos saber si los elementos que observamos en el mismo pertenecen o no a un cierto conjunto de elementos predefinidos. Por ejemplo podemos tener una base de direcciones de email confiables y queremos analizar si un email puede ser spam o no verificando si su emisor esta o no esta en la lista de direcciones confiables.

La <b>base</b> de elementos contra los cuales verificamos es siempre muy grande, no sirve mantenerlas en memoria en un hash o estructura similar.

El metodo de filtrado mas popular consiste en usar los llamados <b>Filtros de Bloom</b>. Un <b>Filtro de Bloom</b> es un vector binario de B bits y k funciones de hashing 0..B. Para agregar un elemento al filtro le aplicamos las funciones de hashing y luego encendemos en 1 los bits apuntados por las funciones. Se prenden k o menos bits segun haya o no colisiones.

Para verificar si un dato del stream pertenece a nuestro conjunto le aplicamos funciones de hashing y verificamos si los bits estan todos en 1. Si alguno esta en 0 entonces el elemento no pertenece al conjunto. Si todos los bits estan en 1 entonces el elemento pertenece al mismo con una cierta probabilidad ya que podria haber un falso positivo si alguno de esos bits fue encendido por algun otro elemento.

Si tenemos <b>k</b> funciones de hashing, <b>n</b> elementos a hashear y <b>m</b> bits. Luego de hashear todos los elementos. Cuantos bits quedan en 1 y cuantos en 0 en el vector? Cual es la probabilidad de un falso positivo?

Comenzamos con algunos calculos basicos:

La probabilidad de que una funcion encienda un cierto bit es <b>1/m</b>. <br>
La probabilidad de que una funcion no encienda un cierto bit es <b>1 - (1/m)</b> <br>
Si tenemos <b>k</b> funciones de hashing la probabilidad de que ninguna encienda un cierto bit es: <br>

\begin{equation}
[1 - (1/m)]^k
\end{equation}

Luego de insertar <b>n</b> elementos la probabilidad de que un cierto bit siga siendo 0 es:

\begin{equation}
[1 - (1/m)]^{kn}
\end{equation}

La probabilidad de que las k posiciones posiciones a testear sean 1 entonces es:

\begin{equation}
(1 - [1 - (1/m)]^{kn})^k
\end{equation}

Y una estimacion para la formula es:

\begin{equation}
(1 - e^{kn/m})^k
\end{equation}

Esta funcion se puede graficar para probar valores de n,m y k que funcionen correctamente.

Para valores de n y m fijos es valor optimo de k se puede calcular de la forma:

\begin{equation}
k = \frac{m}{n} * log(2)
\end{equation}

Para valores de n y m fijos el grafico de k segun segun la probabilidad de falsos positivos es:

<img src="k_vs_falsos_positivos.png">

Como podemos ver la probabilidad de un falso positivo va bajando hasta llegar al optimo y luego sube ya que hay demasiados bits en 1 en el filtro. 

Si usamos el k optimo entonces la cantidad de bits a usar puede calcularse como:

\begin{equation}
m = \frac{n * ln(p)}{ln(2)^2}
\end{equation}

Siendo <b>p</b> la probabilidad que queremos para un falso positivo.

<b>Ejemplo</b>:

Si tenemos mil millones de direcciones de email y queremos que la probabilidad de un falso positivo sea: p = 0.01 entonces:

\begin{equation}
m = -\frac{1000000000 * ln(0.01)}{ln(2)^2} = 9585 000 000
\end{equation}

m = 9585 millones de bits

\begin{equation}
k = \frac{9585 000 000}{1000 000 000} * log(2) = 2.88
\end{equation}

Es decir 3 funciones de hashing.

Algo interesante de los filtros de Bloom es que puede usarse el filtro para estimar la cantidad de elementos que han sido insertados en el mismo.

\begin{equation}
n \approx - \frac{m * ln(1 - x/m)}{k}
\end{equation}

M = Cantidad de bits del filtro <br>
X = Cantidad de bits en 1

Los filtros de Bloom son muy sencillos en su concepto pero aplicarlos correctamente requiere el uso de estas formulas y no todo el mundo las conoce.